# Mendefinisikan Variable Penting

In [83]:
from pandas import read_csv
import pandas as pd

def Preprocessing(data):
    print("Tahap Preprocessing Dimulai")

    # Case Folding
#     data = pd.DataFrame([input], columns=['pertanyaan'])
    data['lower'] = data['pertanyaan'].str.lower()
    print("Tahap Case Folding Berhasil :",data['lower'].iloc[0])

    # Punctual Removal
    data['punctual'] = data['lower'].str.replace('[^a-zA-Z0-9]+', ' ', regex=True)
    print("Tahap Punctual Removal Berhasil :",data['punctual'].iloc[0])

    # Normalization
    kamus_baku = pd.read_csv('kata_baku.csv', sep=";")
    dict_kamus_baku = kamus_baku[['slang', 'baku']].to_dict('list')
    dict_kamus_baku = dict(zip(dict_kamus_baku['slang'], dict_kamus_baku['baku']))
    norm = []
    for i in data['punctual']:
        res = " ".join(dict_kamus_baku.get(x, x) for x in str(i).split())
        norm.append(str(res))
    data['normalize'] = norm
    print("Tahap Normalisasi Berhasil :",data['normalize'].iloc[0])

    # Stopword Removal
    stop_words = set(stopwords.words('indonesian'))
    swr = []
    for i in data['normalize']:
        tokens = word_tokenize(i)
        filtered = [word for word in tokens if word not in stop_words]
        swr.append(" ".join(filtered))
    data['stopwords'] = swr
    print("Tahap Stopword Removal Berhasil :",data['stopwords'].iloc[0])

    # Stemming
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    stem = []
    for i in data['stopwords']:
        stem.append(stemmer.stem(str(i)))
    data['stemmed'] = stem
    print("Tahap Stemming Berhasil :",data['stemmed'].iloc[0])
    return data

def Model_Sequencing(data):
    model_tokenizer = pickle.load(open('model_tokenizer.pkl', 'rb'))
    model_sequence = model_tokenizer.texts_to_sequences(data['stemmed'])
    max_seq_len = pickle.load(open('max_seq_len.pkl', 'rb'))
    model_sequence_padded = pad_sequences(model_sequence, maxlen=max_seq_len) #maxlen mengikuti maxlen model training
    print("Tahap Model Sequencing Berhasil :", model_sequence_padded)
    return model_sequence_padded
try:
    pertanyaan_oov = pd.read_csv("pertanyaan_oov.csv", header=None, names=['pertanyaan', 'jawaban'])
except pd.errors.EmptyDataError:
    print("Belum Ada Pertanyaan OOV")
if (pertanyaan_oov.shape[1] == 1):
    print("Belum Ada Jawaban Atas Pertanyaan OOV")
elif (pertanyaan_oov.shape[1] > 1):
    print("Sudah Ada Jawaban Atas Pertanyaan OOV")
    print("Tahap Penambahan Pertanyaan OOV ke Corpus Dimulai")
    pertanyaan_oov['pertanyaan']
#     pertanyaan_oov = Preprocessing(pertanyaan_oov)
#     pertanyaan_oov['model_sequence_padded'] = Model_Sequencing(pertanyaan_oov)
#     pertanyaan_oov['model_sequence_padded']

Sudah Ada Jawaban Atas Pertanyaan OOV
Tahap Penambahan Pertanyaan OOV ke Corpus Dimulai


In [93]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('indonesian'))
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from keras_preprocessing.sequence import pad_sequences
import pickle
from keras.models import load_model
def OOV_Preprocessing(data):
    # Case Folding
    data['lower'] = data['Pertanyaan'].str.lower()

    # Punctual Removal
    data['punctual'] = data['lower'].str.replace('[^a-zA-Z0-9]+', ' ', regex=True)

    # Normalization
    kamus_baku = pd.read_csv('kata_baku.csv', sep=";")
    dict_kamus_baku = kamus_baku[['slang', 'baku']].to_dict('list')
    dict_kamus_baku = dict(zip(dict_kamus_baku['slang'], dict_kamus_baku['baku']))
    norm = []
    for i in data['punctual']:
        res = " ".join(dict_kamus_baku.get(x, x) for x in str(i).split())
        norm.append(str(res))
    data['normalize'] = norm

    # Stopword Removal
    stop_words = set(stopwords.words('indonesian'))
    swr = []
    for i in data['normalize']:
        tokens = word_tokenize(i)
        filtered = [word for word in tokens if word not in stop_words]
        swr.append(" ".join(filtered))
    data['stopwords'] = swr

    # Stemming
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    stem = []
    for i in data['stopwords']:
        stem.append(stemmer.stem(str(i)))
    data['stemmed'] = stem
    return data

def OOV_Model_Sequences(data):
    model_tokenizer = pickle.load(open('model_tokenizer.pkl', 'rb'))
    model_sequences = model_tokenizer.texts_to_sequences(data['stemmed'])
    max_seq_len = pickle.load(open('max_seq_len.pkl', 'rb'))
    model_sequences_padded = pad_sequences(model_sequences, maxlen=max_seq_len)
    print(model_sequences_padded)
    return model_sequences_padded

def OOV_Corpus_Sequences(data):
    corpus_tokenizer = pickle.load(open('corpus_tokenizer.pkl', 'rb'))
    corpus_sequences = corpus_tokenizer.texts_to_sequences(data['stemmed'])
    return corpus_sequences

def Predict_OOV(model_sequences_padded):
    model = load_model('chatbotmodel.h5')
    categorical_predicted_label = []
    onehot_predicted_label = model.predict(model_sequences_padded)
    for i in range(0, len(model_sequences_padded)):
        categorical_predicted_label.append(onehot_predicted_label[i].argmax())
    return categorical_predicted_label

def Tambah_Pertanyaan_OOV():
    try:
        data = pd.read_csv("pertanyaan_oov.csv", header=None, names=['Pertanyaan', 'Jawaban', 'label'])
    except pd.errors.EmptyDataError:
        print("Belum Ada Pertanyaan OOV")
    if (data.shape[1] == 2):
        print("Belum Ada Jawaban Atas Pertanyaan OOV")
    elif (data.shape[1] > 2):
        print("Sudah Ada Jawaban Atas Pertanyaan OOV")
        print("Tahap Penambahan Pertanyaan OOV ke Corpus Dimulai")
        data = OOV_Preprocessing(data)
        data['model_sequences_padded'] = OOV_Model_Sequences(data)
        data['Sequences'] = OOV_Corpus_Sequences(data)
        data['predicted_label'] = Predict_OOV(data)
        data_IPO = data[['Pertanyaan', 'Jawaban', 'Sequences']].loc[data['predicted_label'] == int(0)].reset_index(drop=True)
        data_Investasi = data[['Pertanyaan', 'Jawaban', 'Sequences']].loc[data['predicted_label'] == int(1)].reset_index(drop=True)
        data_Istilah = data[['Pertanyaan', 'Jawaban', 'Sequences']].loc[data['predicted_label'] == int(2)].reset_index(drop=True)
        data_Strategi = data[['Pertanyaan', 'Jawaban', 'Sequences']].loc[data['predicted_label'] == int(3)].reset_index(drop=True)
        data_Trading = data[['Pertanyaan', 'Jawaban', 'Sequences']].loc[data['predicted_label'] == int(4)].reset_index(drop=True)
        df_Investasi = pickle.load(open('df_Investasi.pkl', 'rb'))
        df_IPO = pickle.load(open('df_IPO.pkl', 'rb'))
        df_Istilah = pickle.load(open('df_Istilah.pkl', 'rb'))
        df_Strategi = pickle.load(open('df_Strategi.pkl', 'rb'))
        df_Trading = pickle.load(open('df_Trading.pkl', 'rb'))
        frames = [[data_IPO, df_IPO],[data_Investasi, df_Investasi],[data_Istilah, df_Istilah],[data_Strategi, df_Strategi],[data_Trading, df_Trading]]
        for list_df in frames:
            list_df[1] = pd.concat(list_df)
        pickle.dump(df_IPO, open('df_IPO.pkl', 'wb'))
        pickle.dump(df_Investasi, open('df_Investasi.pkl', 'wb'))
        pickle.dump(df_Istilah, open('df_Istilah.pkl', 'wb'))
        pickle.dump(df_Strategi, open('df_Strategi.pkl', 'wb'))
        pickle.dump(df_Trading, open('df_Trading.pkl', 'wb'))
        file_intents = './Dataset/Intents.csv'
        df_intents = read_data(file_intents)
        df = pd.concat([df_intents,data])
        df.to_csv('./Dataset/Intents.csv', index=False)
        import os
        os.remove('pertanyaan_oov.csv')
        
Tambah_Pertanyaan_OOV()

Sudah Ada Jawaban Atas Pertanyaan OOV
Tahap Penambahan Pertanyaan OOV ke Corpus Dimulai
[[  0   0   0   0   0   0   0   1]
 [  0   0   0   0   0   0   0 329]
 [  0   0   0   0   0   0   0   1]
 [  0   0   0   0   0   0   0   1]]


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ValueError: Wrong number of items passed 8, placement implies 1

In [84]:
pertanyaan_oov['pertanyaan']

0    a
1    b
2    c
3    d
Name: pertanyaan, dtype: object

In [49]:
import pickle
df_Investasi = pickle.load(open('df_Investasi.pkl', 'rb'))
df_Investasi

,Pertanyaan,Jawaban,Sequences
0,Apa perbedaan antara investasi jangka pendek d...,Investasi jangka pendek biasanya berlangsung d...,"[47, 3, 20, 164, 20]"
1,Bagaimana cara mengelola risiko dalam investas...,Beberapa cara untuk mengelola risiko dalam inv...,"[68, 8, 3, 2]"
2,Apa yang harus diperhatikan saat memilih saham...,Saat memilih saham untuk investasi perhatikan ...,"[69, 21, 2, 3]"
3,Bagaimana cara mengukur kinerja investasi saham?,Kinerja investasi saham dapat diukur dengan me...,"[22, 70, 3, 2]"
4,Bagaimana cara memahami risiko investasi saham?,Memahami risiko investasi saham melibatkan pen...,"[23, 8, 3, 2]"
5,Bagaimana cara mengidentifikasi saham yang pot...,Mengidentifikasi saham yang potensial untuk ja...,"[24, 2, 165, 3, 20]"
6,Bagaimana cara memanfaatkan laporan keuangan p...,Laporan keuangan perusahaan seperti laporan la...,"[167, 16, 19, 11, 4, 3, 2]"
7,Apa yang dimaksud dengan risiko sistemik dalam...,Risiko sistemik adalah risiko yang terkait den...,"[8, 168, 3, 2]"
8,Bagaimana cara menggunakan analisis sentimen d...,Analisis sentimen melibatkan memantau dan meng...,"[4, 106, 3, 2]"
9,Apa yang harus dilakukan jika terjadi kegagala...,Jika terjadi kegagalan dalam investasi saham p...,"[169, 3, 2]"


In [3]:
import requests
requests.get('https://berita-indo-api-next.vercel.app/api/cnbc-news/market').json()

{'message': 'Result of type market news in CNBC News',
 'total': 100,
 'data': [{'title': 'Di Luar Dugaan! Inflasi AS Memanas, Wall Street Dibuka Hijau',
   'link': 'https://www.cnbcindonesia.com/market/20230913204117-17-472282/di-luar-dugaan-inflasi-as-memanas-wall-street-dibuka-hijau',
   'contentSnippet': 'Bursa Wall Street dibuka menghijau padahal inflasi Amerika Serikat lagi memanas. Kok bisa sih?',
   'isoDate': '2023-09-13T13:57:18.000Z',
   'image': {'small': 'https://awsimages.detik.net.id/visual/2023/07/24/ilustrasi-wall-street-apj-david-ake_169.jpeg?w=1200&q=90',
    'large': 'https://awsimages.detik.net.id/visual/2023/07/24/ilustrasi-wall-street-apj-david-ake_169.jpeg?w=1200&q=100'}},
  {'title': 'PSAK 74 Bakal Sedot Modal Asuransi',
   'link': 'https://www.cnbcindonesia.com/market/20230913172807-17-472211/psak-74-bakal-sedot-modal-asuransi',
   'contentSnippet': 'Pelaku industri diharapkan dapat mengimplementasikan PSAK 74 tentang Kontrak Asuransi pada 1 Januari 2025.',
  

In [1]:
from tradingview_ta import TA_Handler, Interval, Exchange
import tradingview_ta

In [5]:
handler = TA_Handler(
    symbol="BBCA",
    exchange="idx",
    screener="indonesia",
    interval=Interval.INTERVAL_1_DAY,
    timeout=None
)
# analysis = handler.get_analysis().summary
# analysis = handler.get_analysis().oscillators
analysis = handler.get_analysis().moving_averages
analysis

{'RECOMMENDATION': 'STRONG_BUY',
 'BUY': 13,
 'SELL': 1,
 'NEUTRAL': 1,
 'COMPUTE': {'EMA10': 'BUY',
  'SMA10': 'SELL',
  'EMA20': 'BUY',
  'SMA20': 'BUY',
  'EMA30': 'BUY',
  'SMA30': 'BUY',
  'EMA50': 'BUY',
  'SMA50': 'BUY',
  'EMA100': 'BUY',
  'SMA100': 'BUY',
  'EMA200': 'BUY',
  'SMA200': 'BUY',
  'Ichimoku': 'NEUTRAL',
  'VWMA': 'BUY',
  'HullMA': 'BUY'}}

In [27]:
# !pip install yahoo_fin
from yahoo_fin import stock_info as si
income_statement = si.get_income_statement("AAPL")
income_statement
# from yahoofinance import BalanceSheet
# req = BalanceSheet('AAPL')
# req

TypeError: string indices must be integers

In [30]:
si.get_quote_table("bbri")

IndexError: list index out of range

In [31]:
from yahoo_fin.stock_info import get_data, tickers_sp500, tickers_nasdaq, tickers_other, get_quote_table, get_balance_sheet

""" pull historical data for Netflix (NFLX) """
get_data("BBRI")


AssertionError: {'chart': {'result': None, 'error': {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}}}

In [22]:
nflx

""


In [12]:
import requests
kode_saham = "TPIA"
key_goAPI = "TbZXoTsXp9Va3NKwVub6JBRjQkFkGi"
# get_indices = requests.get("https://api.goapi.id/v1/stock/idx/indices?api_key="+key_goAPI)
# get_info_saham = requests.get("https://api.goapi.id/v1/stock/idx/"+kode_saham+"?api_key="+key_goAPI)
get_harga_saham = requests.get("https://api.goapi.id/v1/stock/idx/prices?api_key="+key_goAPI+"&symbols="+kode_saham)
# get_semua_list_saham = requests.get("https://api.goapi.id/v1/stock/idx/companies?api_key="+key_goAPI)
# get_history_harga = requests.get("https://api.goapi.id/v1/stock/idx/"+kode_saham+"/historical?api_key="+key_goAPI+"&from='2022-01-17'&to='2022-02-17'")
# get_broker_summary = requests.get("https://api.goapi.id/v1/stock/idx/"+kode_saham+"/broker_summary?api_key="+key_goAPI+"&from='2022-01-17'&to='2022-02-17'")

In [13]:
# indices = get_indices.json()
# data_saham = response.json()['data']['result']
# info_saham = get_info_saham.json()
harga_saham = get_harga_saham.json()
# semua_list_saham = get_semua_list_saham.json()
# history_harga = get_history_harga.json()
# broker_summary = get_broker_summary.json()

In [20]:
harga_saham['data']['results'][0]
# indices
# semua_list_saham['data']['results']
# history_harga
# broker_summary

{'date': '2023-08-31',
 'symbol': 'TPIA',
 'open': '2180',
 'high': '2190',
 'low': '2120',
 'close': '2120',
 'volume': 9334600}

In [11]:
len(requests.get("https://api.goapi.id/v1/stock/idx/prices?api_key="+key_goAPI+"&symbols="+"bba").json()['data']['results'])

0

In [24]:
list_saham = []
for i in range (len(semua_list_saham['data']['results'])):
    list_saham.append(semua_list_saham['data']['results'][i]['ticker'])

In [25]:
# list_saham

In [26]:
dict_info_saham = {}
for kode_saham in list_saham:
    get_info_saham  = requests.get("https://api.goapi.id/v1/stock/idx/"+kode_saham+"?api_key="+key_goAPI)
    info_saham = get_info_saham.json()
#     print(info_saham)
    dict_info_saham[info_saham['data']['result']['ticker']]=info_saham['data']['result']

In [30]:
dict_info_saham['ANTM']

{'ticker': 'ANTM',
 'name': 'Aneka Tambang Tbk.',
 'description': 'PT Aneka Tambang Tbk operates as a diversified mining and metals company in Indonesia. The company operates through three segments: Nickel, Precious Metals and Refinery, and Bauxite and Alumina. It is involved in the exploration, excavation, processing, and marketing of alumia, nickel ore, ferronickel, gold, silver, bauxite, and coal. The company also engages in the construction, trading, industry, agriculture, printing, and ground transportation businesses; manufacture and sale of stainless steel; and provision of industrial area management and heavy equipment rental services. It also exports its products. The company was incorporated in 1968 and is headquartered in Jakarta, Indonesia. PT Aneka Tambang Tbk is a subsidiary of PT Indonesia Asahan Aluminium (Persero).',
 'email': 'corsec@antam.com',
 'sector': 'Barang Baku',
 'phone': '021-7805119; 021-7891234 ; 021-7812635 \r\n',
 'address': 'Gedung. Aneka Tambang \r\nJl

In [32]:
import json
with open("info_saham.json", "w") as outfile:
    json.dump(dict_info_saham, outfile)

In [77]:
requests.get("https://api.goapi.id/v1/stock/idx/"+'AALI'+"?api_key="+key_goAPI).json()

{'status': 'success',
 'message': 'Showing detail information...',
 'data': {'result': {'ticker': 'AALI',
   'name': 'Astra Agro Lestari Tbk',
   'description': 'PT Astra Agro Lestari Tbk, together with its subsidiaries, engages in the palm oil business in Indonesia. It provides crude palm oil and its derivatives, palm kernel and its derivatives, and other products. The company manages 286,877 hectares of oil palm plantations in Sumatra, Kalimantan, and Sulawesi. It also operates palm oil refineries in the Mamuju Utara Regency in West Sulawesi province; and Dumai, Riau province. In addition, the company exports its processed palm oil products, including olein, and stearin to China, Bangladesh, Pakistan, Malaysia, the Philippines, and South Korea. Further, it operates fertilizer blending plants; and engages in the integrated cattle-in-plantation business. The company was founded in 1988 and is headquartered in Jakarta, Indonesia. PT Astra Agro Lestari Tbk is a subsidiary of PT Astra Int

In [3]:
dict_info_saham

{'ticker': 'ANTM',
 'name': 'Aneka Tambang Tbk.',
 'description': 'PT Aneka Tambang Tbk operates as a diversified mining and metals company in Indonesia. The company operates through three segments: Nickel, Precious Metals and Refinery, and Bauxite and Alumina. It is involved in the exploration, excavation, processing, and marketing of alumia, nickel ore, ferronickel, gold, silver, bauxite, and coal. The company also engages in the construction, trading, industry, agriculture, printing, and ground transportation businesses; manufacture and sale of stainless steel; and provision of industrial area management and heavy equipment rental services. It also exports its products. The company was incorporated in 1968 and is headquartered in Jakarta, Indonesia. PT Aneka Tambang Tbk is a subsidiary of PT Indonesia Asahan Aluminium (Persero).',
 'email': 'corsec@antam.com',
 'sector': 'Barang Baku',
 'phone': '021-7805119; 021-7891234 ; 021-7812635 \r\n',
 'address': 'Gedung. Aneka Tambang \r\nJl

In [82]:
dict_saham_per_sector = {}
for kode_saham in list_saham:
    get_info_saham  = requests.get("https://api.goapi.id/v1/stock/idx/"+kode_saham+"?api_key="+key_goAPI)
    info_saham = get_info_saham.json()
#     print(info_saham)
    if(info_saham['data']['result']['sector'] not in dict_saham_per_sector):
        dict_saham_per_sector[info_saham['data']['result']['sector']] = []
    dict_saham_per_sector[info_saham['data']['result']['sector']].append(kode_saham)

In [31]:
# dict_saham_per_sector

In [20]:
import json
import translators as ts
with open('info_saham.json', 'r') as open_info_saham:
    # Reading from json file
    json_info_saham = json.load(open_info_saham) #info semua saham
    
kodes = json_info_saham.keys()
for kode in kodes:
    if(json_info_saham[kode]['description'] == None):
        continue
    else:
        json_info_saham[kode]['description'] = ts.translate_text(json_info_saham[kode]['description'], translator='google', to_language='id')

with open("info_saham1.json", "w") as outfile:
    json.dump(json_info_saham, outfile)

In [84]:
import json
with open("saham_per_sector.json", "w") as outfile:
    json.dump(dict_saham_per_sector, outfile)

In [34]:
harga_saham

{'status': 'success',
 'message': 'Menampilkan harga terakhir dari 1 saham.',
 'data': {'results': [{'date': '2023-08-08',
    'symbol': 'BBCA',
    'open': '9275',
    'high': '9300',
    'low': '9250',
    'close': '9275',
    'volume': 17697000}]}}

In [56]:
import translators as ts
data_saham['description'] = ts.translate_text(data_saham['description'], translator='google', to_language='id')
data_saham

{'ticker': 'ANTM',
 'name': 'Aneka Tambang Tbk.',
 'description': 'PT Aneka Tambang TBK beroperasi sebagai perusahaan pertambangan dan logam yang beragam di Indonesia. Perusahaan beroperasi melalui tiga segmen: nikel, logam mulia dan kilang, dan bauksit dan alumina. Ini terlibat dalam eksplorasi, penggalian, pemrosesan, dan pemasaran alumia, bijih nikel, ferronickel, emas, perak, bauksit, dan batubara. Perusahaan juga terlibat dalam bisnis konstruksi, perdagangan, industri, pertanian, pencetakan, dan transportasi darat; pembuatan dan penjualan baja tahan karat; dan penyediaan manajemen kawasan industri dan layanan penyewaan alat berat. Ini juga mengekspor produknya. Perusahaan ini didirikan pada tahun 1968 dan berkantor pusat di Jakarta, Indonesia. Pt Aneka Tambang TBK adalah anak perusahaan Pt Indonesia Asahan Aluminium (Persero).',
 'email': 'corsec@antam.com',
 'sector': 'Barang Baku',
 'phone': '021-7805119; 021-7891234 ; 021-7812635 \r\n',
 'address': 'Gedung. Aneka Tambang \r\nJl

In [ ]:
import goslate
# deskripsi = data_saham['description']
gs = goslate.Goslate()
test = gs.translate("hello", 'id')

In [1]:
# Variable Penting
ListLabelUnique = []
ListLabelSpesifikUnique = []
testSize = 0.15 #Pembagian ukuran datatesing

MAX_NB_WORDS = 100000 #Maximum jumlah kata pada vocabulary yang akan dibuat
max_seq_len = 6 #Panjang kalimat maximum

num_epochs = 400 #Jumlah perulangan / epoch saat proses training

In [4]:
# !pip3 install fuzzywuzzy

In [6]:
from fuzzywuzzy import process, fuzz
word_index = ['saham','jual','beli','raksadana','dividen']
test = process.extract('deviden', word_index, scorer=fuzz.token_sort_ratio)[0][0]
print(test)

dividen


# Import Module

In [2]:
# Import CSV
import csv
# Import json
import json
# Import Pandas
import pandas as pd
# Settingan di Pandas untuk menghilangkan warning
pd.options.mode.chained_assignment = None  # default='warn'

# Import Numpy
import numpy as np

# Loading Bar TQDM
from tqdm import tqdm

# Stopword Removal
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

# Stemming (Sastrawi)
# !pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Tokenizer
from keras.preprocessing.text import Tokenizer

# pad_sequences untuk google colab
# from keras.utils import pad_sequences
# pad_sequences untuk jupter-lab
from keras_preprocessing.sequence import pad_sequences

# Pickle FastText
import pickle

# Split Data
from sklearn.model_selection import train_test_split

# Label Encoder
from sklearn.preprocessing import LabelEncoder

# Model Building
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, Dropout, LSTM, Dense
# from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D
from tensorflow.keras.backend import clear_session
from keras.models import load_model

# Callbacks
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

# Grafik
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
np.random.seed(0)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
stop_words = set(stopwords.words('indonesian'))
swr = []
tokens = word_tokenize("bagaimana beli saham")
filtered = [word for word in tokens if word not in stop_words]
swr.append(" ".join(filtered))
print(swr)
factory = StemmerFactory()
stemmer = factory.create_stemmer()
print(stemmer.stem(" beli saham"))

['beli saham']
beli saham


# Function Mengubah File CSV menjadi File JSON

In [4]:
def csv_to_json(CsvPath, JsonPath):
    JsonArray = []
    UniqueLabel = {}
    
    # Membuka File CSV 
    with open(CsvPath, encoding='utf-8') as CsvFile: 
        # Membaca file csv dan menjadikannya dict menggunakan module DictReader
        CsvReader = csv.DictReader(CsvFile)

        for row in CsvReader: 
            # Memasukkan tiap baris dict ke dalam array
            
            # Menambahkan label spesifik
            if(row['Label'] not in UniqueLabel):
                UniqueLabel[row['Label']]=1
                ListLabelUnique.append(row['Label'])
            else:
                UniqueLabel[row['Label']]+=1
                            
            row['Label_Spesifik']=row['Label']+"_"+str(UniqueLabel[row['Label']])
            row['Label_Spesifik_IntegerEncoded']=int(UniqueLabel[row['Label']])
            # print(row['LabelSpesifik'])
            JsonArray.append(row)
            
                            
    # Memasukkan dict ke dalam json
    with open(JsonPath, 'w', encoding='utf-8') as JsonFile: 
        JsonString = json.dumps(JsonArray, indent=4)
        JsonFile.write(JsonString)

# Memanggil Function csv_to_json

In [5]:
csvFilePath = r'D:\Kuliah\File Kuliah\Skripsi\Program Chabot\StockEdu\Dataset\Intents.csv'
jsonFilePath = r'D:\Kuliah\File Kuliah\Skripsi\Program Chabot\StockEdu\Dataset\Intents.json'
csv_to_json(csvFilePath, jsonFilePath)

# Function Membaca file JSON menjadi DataFrame

In [6]:
def read_data(filename):
    # Membaca file CSV ke dalam Dataframe
    df = pd.read_json(filename) 
    # Encoding Categorical Data (Mengubah data kategorikal menjadi angka)
    LE = LabelEncoder()
    df['Label_IntegerEncoded'] = LE.fit_transform(df['Label'])

    # Membagi dataframe menjadi data training & testing
#     df_training, df_testing = train_test_split(df, test_size=testSize, random_state=42, shuffle=True)

    # Reset Index
#     df_training = df_training.reset_index()
#     df_testing = df_testing.reset_index()

#     return df_training, df_testing
    return df

# Memanggil Function read_data

In [7]:
# Memanggil Function Read Data
FileIntents = './Dataset/Intents.json'
df = read_data(FileIntents)
# pd.set_option("max_rows", None)

In [8]:
# Proses One Hot Encoded, Tidak perlu lagi karena sudah digantikan oleh pd.get_dummies[df['label']].values

# from sklearn.preprocessing import OneHotEncoder

# y = OneHotEncoder(sparse=False)
# y = list(y.fit_transform(df[['Label']]))
# df['Label_OneHotEncoded'] = y
# df
# df_y = pd.DataFrame(y)

In [9]:
# Reverse One Hot (Multi Class)
print(np.argmax([0,0,0,0,1]))
print(np.argmax([0.1,0.5,0.6,0.5,0.1]))

4
2


In [10]:
# Reverse One Hot (Multi Label)
print(np.where([[1,0,0,0,1]]))
print(np.where([[1,0,0,0,1]])[1])
print(np.where([[1,0,0,0,1]])[1][0],np.where([[1,0,0,0,1]])[1][1])

(array([0, 0], dtype=int64), array([0, 4], dtype=int64))
[0 4]
0 4


In [11]:
ListLabelSpesifikUnique = list(set(df['Label_Spesifik_IntegerEncoded']))
print("List Label Unik :",ListLabelUnique)
print("List Label Spesifik Unik :",ListLabelSpesifikUnique)

List Label Unik : ['Istilah', 'Strategi', 'Investasi', 'Sekuritas', 'Broker']
List Label Spesifik Unik : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [12]:
# Proses dibalik Encoder Label
print(list(set(df['Label'])))
LE = LabelEncoder()
print(LE.fit_transform(list(set(df['Label']))))

['Broker', 'Investasi', 'Strategi', 'Sekuritas', 'Istilah']
[0 1 4 3 2]


In [13]:
# Proses dibalik Encoder Label Spesifik
df[['Label_Spesifik','Label_Spesifik_IntegerEncoded']]

,Label_Spesifik,Label_Spesifik_IntegerEncoded
0,Istilah_1,1
1,Istilah_2,2
2,Istilah_3,3
3,Istilah_4,4
4,Istilah_5,5
...,...,...
82,Broker_12,12
83,Broker_13,13
84,Broker_14,14
85,Broker_15,15


In [14]:
# Alasan tidak menggunakan stopwords
stop_words = set(stopwords.words('indonesian'))
swr = []
for i in tqdm(["bagaimana beli saham","kapan beli saham","mengapa membeli saham"]):
    tokens = word_tokenize(i)
    filtered = [word for word in tokens if word not in stop_words]
    swr.append(" ".join(filtered))
swr

100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 2999.50it/s]


['beli saham', 'beli saham', 'membeli saham']

In [15]:
def preprocessing(data):
    # Case Folding
    data['lower'] = data['Pertanyaan'].str.lower()
    
    # Punctual Removal
    data['punctual'] = data['lower'].str.replace('[^a-zA-Z]+',' ', regex=True)
    
    # Normalization
    kamus_baku = pd.read_csv('kata_baku.csv', sep=";")
    dict_kamus_baku = kamus_baku[['slang','baku']].to_dict('list')
    dict_kamus_baku = dict(zip(dict_kamus_baku['slang'], dict_kamus_baku['baku']))
    norm = []
    for i in data['punctual']:
        res = " ".join(dict_kamus_baku.get(x, x) for x in str(i).split())
        norm.append(str(res))
    data['normalize'] = norm
    
    # Stopword Removal
#     stop_words = set(stopwords.words('indonesian'))
#     swr = []
#     for i in tqdm(data['normalize']):
#         tokens = word_tokenize(i)
#         filtered = [word for word in tokens if word not in stop_words]
#         swr.append(" ".join(filtered))
#     data['stopwords'] = swr
    
    # Stemming
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    stem = []
    for i in tqdm(data['normalize']):
#     for i in tqdm(data['stopwords']):
        stem.append(stemmer.stem(str(i)))
    data['stemmed'] = stem
    
    return data

In [16]:
preprocessing(df)

100%|██████████████████████████████████████████████████████████████████████████████████| 87/87 [00:04<00:00, 19.74it/s]


,Pertanyaan,Jawaban,Label,,Istilah,Label_Spesifik,Label_Spesifik_IntegerEncoded,Label_IntegerEncoded,lower,punctual,normalize,stemmed
0,Apa itu saham?,Saham adalah instrumen keuangan yang mewakili ...,Istilah,,TRUE,Istilah_1,1,2,apa itu saham?,apa itu saham,apa itu saham,apa itu saham
1,Bagaimana cara membeli saham?,Anda dapat membeli saham melalui perusahaan se...,Istilah,,FALSE,Istilah_2,2,2,bagaimana cara membeli saham?,bagaimana cara membeli saham,bagaimana cara membeli saham,bagaimana cara beli saham
2,Bagaimana cara menjual saham?,Untuk menjual saham Anda dapat menggunakan pla...,Istilah,,FALSE,Istilah_3,3,2,bagaimana cara menjual saham?,bagaimana cara menjual saham,bagaimana cara menjual saham,bagaimana cara jual saham
3,Apa perbedaan antara investasi jangka pendek d...,Investasi jangka pendek biasanya berlangsung d...,Istilah,,TRUE,Istilah_4,4,2,apa perbedaan antara investasi jangka pendek d...,apa perbedaan antara investasi jangka pendek d...,apa perbedaan antara investasi jangka pendek d...,apa beda antara investasi jangka pendek dan ja...
4,Apa itu dividen?,Dividen adalah pembagian sebagian dari laba pe...,Istilah,,TRUE,Istilah_5,5,2,apa itu dividen?,apa itu dividen,apa itu dividen,apa itu dividen
...,...,...,...,...,...,...,...,...,...,...,...,...
82,Bagaimana cara mengenali saham yang mengalami ...,Mengenali saham yang mengalami breakout meliba...,Broker,,FALSE,Broker_12,12,0,bagaimana cara mengenali saham yang mengalami ...,bagaimana cara mengenali saham yang mengalami ...,bagaimana cara mengenali saham yang mengalami ...,bagaimana cara nali saham yang alami breakout
83,Bagaimana cara mengenali saham yang mengalami ...,Mengenali saham yang mengalami breakout meliba...,Broker,,FALSE,Broker_13,13,0,bagaimana cara mengenali saham yang mengalami ...,bagaimana cara mengenali saham yang mengalami ...,bagaimana cara mengenali saham yang mengalami ...,bagaimana cara nali saham yang alami breakout
84,Apa yang dimaksud dengan rasio Price to Earnin...,Rasio Price to Earnings (P/E) adalah rasio yan...,Broker,,TRUE,Broker_14,14,0,apa yang dimaksud dengan rasio price to earnin...,apa yang dimaksud dengan rasio price to earnin...,apa yang dimaksud dengan rasio harga to earnin...,apa yang maksud dengan rasio harga to earnings...
85,Bagaimana cara menggunakan stop-loss dalam tra...,Stop-loss adalah perintah jual yang ditempatka...,Broker,,FALSE,Broker_15,15,0,bagaimana cara menggunakan stop-loss dalam tra...,bagaimana cara menggunakan stop loss dalam tra...,bagaimana cara menggunakan stop loss dalam tra...,bagaimana cara guna stop loss dalam trading saham


In [17]:
# Mengecek panjang kalimat maksimum
longest_string = max(df['stemmed'].values.tolist(), key=len)
max_seq_len = len(longest_string.split())
print(longest_string)
print(max_seq_len)

apa yang maksud dengan stock split dan bagaimana pengaruh hadap harga saham
12


In [18]:
def tokenize(data):
    global tokenizer #Menggunakan variabel global agar 'tokenizer' bisa dipake di luar fungsi ini
    tokenizer = Tokenizer(num_words = MAX_NB_WORDS, char_level=False)
    tokenizer.fit_on_texts(data['stemmed'])
    word_index = tokenizer.word_index
    # Integer Tokenizer
    train_sequences = tokenizer.texts_to_sequences(data['stemmed'])
    # Padding
    word_seq = pad_sequences(train_sequences, maxlen = max_seq_len)

    return word_index, word_seq

In [19]:
word_index, word_seq = tokenize(df)
word_seq[0]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  3, 11,  1])

In [20]:
# Pendefenisian fungsi word embedding

def word_embedding():
    fasttext_word_to_index = pickle.load(open("fasttext_voc", 'rb'))
    
    words_not_found = []
    nb_words = min(MAX_NB_WORDS, len(word_index)+1) # Word_index berasal dari tokenize

    embed_dim = 300 # dimensi matrix (dari fastTextnya cc.id.300.vec)
    embedding_matrix = np.zeros((nb_words, embed_dim)) # Membuat array vector berisi 0 (135 baris kebawah,300 dimension kesamping)
    # Hanya 134 yang akan terisi (dari index 1 sampai 134) untuk index ke 0 hanya akan berisi 300 buah dimension bernilai 0

    for word, index in word_index.items():
        if index < nb_words:
            embedding_vector = fasttext_word_to_index.get(word) # mengambil vector word yang dimasukkan dari word_index dari fasttext_voc
            if (embedding_vector is not None) and len(embedding_vector) > 0:
                embedding_matrix[index] = embedding_vector # memasukkan vector dari word yang didapatkan ke dalam index matrix yang sama dengan index dari word_index
            else:
                words_not_found.append(word)
    
    return embedding_matrix, nb_words, embed_dim

In [21]:
# Pemanggilan Fungsi word embedding
embedding_matrix, nb_words, embed_dim = word_embedding()

In [22]:
embedding_matrix

array([[ 0.    ,  0.    ,  0.    , ...,  0.    ,  0.    ,  0.    ],
       [ 0.0033,  0.089 , -0.1178, ..., -0.0449,  0.038 ,  0.1004],
       [ 0.0128, -0.027 ,  0.038 , ..., -0.0487, -0.014 ,  0.1242],
       ...,
       [ 0.0055,  0.0162, -0.0156, ..., -0.0987,  0.0103,  0.0236],
       [ 0.0392,  0.0414, -0.0187, ...,  0.0347, -0.017 ,  0.0685],
       [-0.0964,  0.0701, -0.0402, ..., -0.0172,  0.0468,  0.1301]])

In [23]:
nb_words

135

In [24]:
embed_dim

300

In [25]:
# Penentuan X (input) dan Y (output)
X_train = word_seq
Y_train = pd.get_dummies(df['Label']).values
Y_train[0:5]
# len(set(Y_train))
# set(Y_train)

array([[0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0]], dtype=uint8)

In [26]:
model = keras.Sequential([
        keras.layers.Embedding(nb_words, embed_dim ,input_length=max_seq_len, 
                               weights=[embedding_matrix], trainable=False),
#         keras.layers.Conv1D(256, 5, padding='same', activation='relu'),
#         keras.layers.MaxPooling1D(pool_size=4),
#         keras.layers.Dropout(0.5),
#         keras.layers.LSTM(10),
#         keras.layers.Dense(1, activation = 'softmax') #softmax
        keras.layers.LSTM(64, return_sequences = True),
        keras.layers.LSTM(32),# returns a sequence of vectors of dimension 32
#         keras.layers.LSTM(32, return_sequences=True), # returns a sequence of vectors of dimension 32
#         keras.layers.LSTM(10),  # return a single vector of dimension 32
        keras.layers.Dense(5, activation='softmax')
    ])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 12, 300)           40500     
                                                                 
 lstm (LSTM)                 (None, 12, 64)            93440     
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dense (Dense)               (None, 5)                 165       
                                                                 
Total params: 146,521
Trainable params: 106,021
Non-trainable params: 40,500
_________________________________________________________________


In [27]:
# model.compile(
#     optimizer = 'adam',
#     loss = 'sparse_categorical_crossentropy',
#     metrics = ['accuracy']
# )

# Integer Label Encoded
# model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# One Hot Label Encoded
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# history_aspek = model.fit(X_train, Y_train,epochs = 400)

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='auto', patience=3)
Prediksi_Label = model.fit(X_train, Y_train,
                        epochs = 400, 
#                         callbacks = [es],
#                         validation_split=0.2,
                        verbose = True # Verbose = 0 (tidak nampak progress), 1/True (progress bar), 2 (angka)
)

Epoch 1/400
3/3 [==============================] - 4s 18ms/step - loss: 1.6107 - accuracy: 0.1609
Epoch 2/400
3/3 [==============================] - 0s 10ms/step - loss: 1.6022 - accuracy: 0.2414
Epoch 3/400
3/3 [==============================] - 0s 17ms/step - loss: 1.5969 - accuracy: 0.2644
Epoch 4/400
3/3 [==============================] - 0s 15ms/step - loss: 1.5923 - accuracy: 0.2414
Epoch 5/400
3/3 [==============================] - 0s 16ms/step - loss: 1.5879 - accuracy: 0.2644
Epoch 6/400
3/3 [==============================] - 0s 17ms/step - loss: 1.5822 - accuracy: 0.3103
Epoch 7/400
3/3 [==============================] - 0s 14ms/step - loss: 1.5749 - accuracy: 0.2759
Epoch 8/400
3/3 [==============================] - 0s 13ms/step - loss: 1.5703 - accuracy: 0.2759
Epoch 9/400
3/3 [==============================] - 0s 12ms/step - loss: 1.5618 - accuracy: 0.2644
Epoch 10/400
3/3 [==============================] - 0s 14ms/step - loss: 1.5535 - accuracy: 0.2759
Epoch 11/400
3/3 [=

In [28]:
# Predict test data
test = model.predict(word_seq)

3/3 [==============================] - 1s 12ms/step


In [29]:
# Convert Predicted array 
array = []
for i in range(0,87):
    array.append(test[i].argmax())

In [30]:
pd.set_option("max_rows", None)
df['Predicted'] = array
df[['Label_IntegerEncoded','Predicted']].loc[df['Label_IntegerEncoded'] != df['Predicted']].head()

,Label_IntegerEncoded,Predicted
48,1,2
49,1,0
58,3,0
64,3,2
66,3,0


In [31]:
df.iloc[12]

Pertanyaan                       Apa yang dimaksud dengan IPO (Initial Public O...
Jawaban                          IPO adalah proses di mana perusahaan pertama k...
Label                                                                      Istilah
                                                                                  
Istilah                                                                       TRUE
Label_Spesifik                                                          Istilah_13
Label_Spesifik_IntegerEncoded                                                   13
Label_IntegerEncoded                                                             2
lower                            apa yang dimaksud dengan ipo (initial public o...
punctual                         apa yang dimaksud dengan ipo initial public of...
normalize                        apa yang dimaksud dengan ipo initial public of...
stemmed                          apa yang maksud dengan ipo initial public offe...
Pred

In [32]:
df[['Label_IntegerEncoded','Predicted']].loc[df['Label_IntegerEncoded'] != df['Predicted']].shape

(7, 2)

In [33]:
Model_Spesifik = keras.Sequential([
        keras.layers.Embedding(nb_words, embed_dim ,input_length=max_seq_len, 
                               weights=[embedding_matrix], trainable=False),
#         keras.layers.Conv1D(256, 5, padding='same', activation='relu'),
#         keras.layers.MaxPooling1D(pool_size=4),
#         keras.layers.Dropout(0.5),
#         keras.layers.LSTM(10),
#         keras.layers.Dense(1, activation = 'softmax') #softmax
        keras.layers.LSTM(64),  # returns a sequence of vectors of dimension 32
#         keras.layers.LSTM(32, return_sequences=True), # returns a sequence of vectors of dimension 32
#         keras.layers.LSTM(10),  # return a single vector of dimension 32
        keras.layers.Dense(87, activation='softmax')
    ])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 12, 300)           40500     
                                                                 
 lstm (LSTM)                 (None, 12, 64)            93440     
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dense (Dense)               (None, 5)                 165       
                                                                 
Total params: 146,521
Trainable params: 106,021
Non-trainable params: 40,500
_________________________________________________________________


In [34]:
Y_train = pd.get_dummies(df['Label_Spesifik']).values
Y_train[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=uint8)

In [35]:
# Integer Label Encoded
Model_Spesifik.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# One Hot Label Encoded
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# history_aspek = model.fit(X_train, Y_train,epochs = 400)

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='auto', patience=3)
Prediksi_Label = Model_Spesifik.fit(X_train, Y_train,
                        epochs = 400, 
#                         callbacks = [es],
                        validation_split=0.2,
                        verbose = True # Verbose = 0 (tidak nampak progress), 1/True (progress bar), 2 (angka)
)

Epoch 1/400
3/3 [==============================] - 2s 301ms/step - loss: 4.4682 - accuracy: 0.0000e+00 - val_loss: 4.4788 - val_accuracy: 0.0000e+00
Epoch 2/400
3/3 [==============================] - 0s 24ms/step - loss: 4.4525 - accuracy: 0.0000e+00 - val_loss: 4.4888 - val_accuracy: 0.0000e+00
Epoch 3/400
3/3 [==============================] - 0s 24ms/step - loss: 4.4425 - accuracy: 0.0435 - val_loss: 4.5002 - val_accuracy: 0.0000e+00
Epoch 4/400
3/3 [==============================] - 0s 26ms/step - loss: 4.4320 - accuracy: 0.0435 - val_loss: 4.5132 - val_accuracy: 0.0000e+00
Epoch 5/400
3/3 [==============================] - 0s 24ms/step - loss: 4.4216 - accuracy: 0.0435 - val_loss: 4.5275 - val_accuracy: 0.0000e+00
Epoch 6/400
3/3 [==============================] - 0s 24ms/step - loss: 4.4115 - accuracy: 0.0435 - val_loss: 4.5453 - val_accuracy: 0.0000e+00
Epoch 7/400
3/3 [==============================] - 0s 23ms/step - loss: 4.3985 - accuracy: 0.0435 - val_loss: 4.5646 - val_accu

In [36]:
test = Model_Spesifik.predict(word_seq)

3/3 [==============================] - 0s 8ms/step


In [37]:
# Convert Predicted array 
array = []
for i in range(0,87):
    array.append(test[i].argmax())

In [45]:
pd.set_option("max_rows", None)
df['Label_Spesifik_Pure_Encoded'] = pd.get_dummies(df['Label_Spesifik']).values.tolist()
df['Predicted'] = array
menang = df[['Label_Spesifik_Pure_Encoded','Predicted']].loc[df['Label_Spesifik_Pure_Encoded'] != df['Predicted']]

In [46]:
menang.shape

(87, 2)

In [47]:
menang

,Label_Spesifik_Pure_Encoded,Predicted
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",35
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",45
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",46
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",47
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",48
5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",49
6,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",50
7,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",51
8,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",52
9,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",36


In [41]:
test1 = Model_Spesifik.predict([['']])

UnimplementedError: Graph execution error:

Detected at node 'sequential_1/Cast' defined at (most recent call last):
    File "c:\users\user\appdata\local\programs\python\python39\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\users\user\appdata\local\programs\python\python39\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\users\user\appdata\local\programs\python\python39\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "c:\users\user\appdata\local\programs\python\python39\lib\site-packages\traitlets\config\application.py", line 845, in launch_instance
      app.start()
    File "c:\users\user\appdata\local\programs\python\python39\lib\site-packages\ipykernel\kernelapp.py", line 612, in start
      self.io_loop.start()
    File "c:\users\user\appdata\local\programs\python\python39\lib\site-packages\tornado\platform\asyncio.py", line 199, 